## Part 5
### Implementation of a second order HMM and decoding with Viterbi

In [1]:
def load_original_train():
    with open("train", encoding="utf-8") as file:
        train_list = file.readlines()
        train_list = [x.strip() for x in train_list]
        return train_list
    
train_list = load_original_train()

In [2]:
annotationDict = {"O": 0, "I-positive": 0, "B-positive": 0, "I-negative":0, "B-negative": 0, "I-neutral":0, "B-neutral":0}

for i in range(len(train_list)):
    annotated_word = train_list[i]
    each_word = annotated_word.split(" ")
    if (len(each_word) == 2):
        if each_word[1] == "O":
            annotationDict["O"] += 1
        elif each_word[1] == "I-positive":
            annotationDict["I-positive"] += 1
        elif each_word[1] == "B-positive":
            annotationDict["B-positive"] += 1
        elif each_word[1] == "I-negative":
            annotationDict["I-negative"] += 1
        elif each_word[1] == "B-negative":
            annotationDict["B-negative"] += 1
        elif each_word[1] == "I-neutral":
            annotationDict["I-neutral"] += 1
        elif each_word[1] == "B-neutral":
            annotationDict["B-neutral"] += 1

print(annotationDict)

{'O': 24242, 'I-positive': 607, 'B-positive': 1208, 'I-negative': 133, 'B-negative': 382, 'I-neutral': 23, 'B-neutral': 65}


In [3]:
def load_files():
    with open("dev.in", encoding="utf-8") as file:
        dev_in_list = file.readlines()
        dev_in_list = [x.strip() for x in dev_in_list]
        return dev_in_list
    
def load_modified_train_file():
    with open("modified_train", encoding="utf-8") as file:
        modified_train_list = file.readlines()
        modified_train_list = [x.strip() for x in modified_train_list]
        return modified_train_list
        
def split_into_sentences(dev_in_list):
    sentences = []
    each_sentence = []
    for word in dev_in_list:
        if word == "":
            sentences.append(each_sentence)
            each_sentence = []
        else:
            each_sentence.append(word)
        
    return sentences

modified_train_list = load_modified_train_file()   
dev_in_list = load_files()
sentences = split_into_sentences(dev_in_list)

In [5]:
def count_word_tag_pair_dict():
    word_tag_pair_dict= {}
    for word_tag_pair in modified_train_list:
        if word_tag_pair_dict.get(word_tag_pair) == None:
            word_tag_pair_dict[word_tag_pair] = 1
        else:
            word_tag_pair_dict[word_tag_pair] += 1
    return word_tag_pair_dict

def get_word_dict():
    modifiedWordDict = {}        
    for i in range(len(modified_train_list)):
        annotated_word = modified_train_list[i]
        each_word = annotated_word.split(" ")
        if each_word[0] in modifiedWordDict:
            modifiedWordDict[each_word[0]] += 1
        else:
            modifiedWordDict[each_word[0]] = 1

    return modifiedWordDict
    

def store_estimate_emission_fix():
    giantEmissionDict = {}
    for word_tag_pair in modified_train_list:
        if word_tag_pair != "":
            word_tag_pair_list = word_tag_pair.split(" ")
            countTop = wordTagPairDict[word_tag_pair]
            countBottom = annotationDict[word_tag_pair_list[len(word_tag_pair_list)-1]]
            emission = countTop/countBottom
            if giantEmissionDict.get(word_tag_pair) == None:
                giantEmissionDict[word_tag_pair] = countTop/countBottom
    return giantEmissionDict

def store_estimate_transition():
    giantTransitionDict = {}
    finalGiantTransitionDict = {}
    for taggedEachTweet in giantTaggedEachTweet:
        for i in range(len(taggedEachTweet) - 1):
            if giantTransitionDict.get(taggedEachTweet[i]+" "+taggedEachTweet[i+1]) == None:
                giantTransitionDict[taggedEachTweet[i]+" "+taggedEachTweet[i+1]] = 1
            else:
                giantTransitionDict[taggedEachTweet[i]+" "+taggedEachTweet[i+1]] += 1
    for eachTransition in giantTransitionDict:
        eachTransitionList = eachTransition.split(" ")
        startState = eachTransitionList[0]
        countTop = giantTransitionDict[eachTransition]
        countBottom = totalStateNumber[startState]
        transitionParam = countTop/countBottom
        finalGiantTransitionDict[eachTransition] = transitionParam
        
    return finalGiantTransitionDict

def separate_tweets():
    countStart = 1
    # counting starts
    giant_each_tweet = []
    each_tweet = []
    totalStateNumber = {}
    for i in range(len(modified_train_list)):
        annotated_word = modified_train_list[i]
        each_word = annotated_word.split(" ")
        if each_word == ['']:
            countStart += 1
            giant_each_tweet.append(each_tweet)
            each_tweet = []

        else:
            each_tweet.append(each_word[len(each_word)-1])
    for i in range(len(giant_each_tweet)):
        giant_each_tweet[i].insert(0, 'START')
        giant_each_tweet[i].append('STOP')
    
    for each_tweet in giant_each_tweet:
        for tag in each_tweet:
            if totalStateNumber.get(tag) == None:
                totalStateNumber[tag] = 1
            else:
                totalStateNumber[tag] += 1

    return giant_each_tweet, countStart, totalStateNumber

modifiedWordDict = get_word_dict()
wordTagPairDict = count_word_tag_pair_dict()
giantEmissionDict = store_estimate_emission_fix()
giantTaggedEachTweet, countStart, totalStateNumber = separate_tweets()
giantTransitionDict = store_estimate_transition()

In [15]:
def store_second_order_transition():
    transitionCount = {}
    startStateCount = {}
    giantSecondTransitionDict = {}        
    for taggedEachTweet in giantTaggedEachTweet:
        for i in range(len(taggedEachTweet) - 1):
            if startStateCount.get(taggedEachTweet[i]+" "+taggedEachTweet[i+1]) == None:
                startStateCount[taggedEachTweet[i]+" "+taggedEachTweet[i+1]] = 1
            else:
                startStateCount[taggedEachTweet[i]+" "+taggedEachTweet[i+1]] += 1
                
        for i in range(len(taggedEachTweet) - 2):
            if transitionCount.get(taggedEachTweet[i]+" "+taggedEachTweet[i+1] + " " + taggedEachTweet[i+2]) == None:
                transitionCount[taggedEachTweet[i]+" "+taggedEachTweet[i+1]+ " "+ taggedEachTweet[i+2]] = 1
            else:
                transitionCount[taggedEachTweet[i]+" "+taggedEachTweet[i+1]+ " "+ taggedEachTweet[i+2]] += 1
    for eachTransition in transitionCount:
        eachTransitionList = eachTransition.split(" ") #START O O
        startState = eachTransitionList[0] + " " + eachTransitionList[1] # START O
        countTop = transitionCount[eachTransition]
        countBottom = startStateCount[startState]
        transitionParam = countTop/countBottom
        giantSecondTransitionDict[eachTransition] = transitionParam

    return giantSecondTransitionDict

giantSecondTransitionDict = store_second_order_transition()
print(giantSecondTransitionDict)

{'START O O': 0.7755681818181818, 'O O O': 0.8720913496137792, 'O O STOP': 0.07618864846711126, 'START B-neutral O': 0.9, 'B-neutral O O': 0.9019607843137255, 'START O B-positive': 0.1653409090909091, 'O B-positive O': 0.6900532859680284, 'B-positive O O': 0.7884615384615384, 'O O B-negative': 0.013625677685553903, 'O B-negative O': 0.7811634349030471, 'B-negative O STOP': 0.12416107382550336, 'O O B-positive': 0.03660701434534376, 'B-positive O STOP': 0.15264423076923078, 'START O B-negative': 0.040340909090909094, 'B-negative O O': 0.8624161073825504, 'O B-negative I-negative': 0.21052631578947367, 'B-negative I-negative I-negative': 0.3875, 'I-negative I-negative O': 0.5849056603773585, 'I-negative O O': 0.8, 'O O B-neutral': 0.0014873098882118697, 'O B-neutral O': 0.7636363636363637, 'O B-positive I-positive': 0.29573712255772644, 'B-positive I-positive O': 0.6083333333333333, 'I-positive O O': 0.7549295774647887, 'START O B-neutral': 0.013068181818181817, 'B-positive I-positive I-

In [16]:
def second_order_viterbi(sentence):
    all_tags=["O","B-negative","B-positive","B-neutral","I-negative","I-positive","I-neutral"]
    sentence.append("") #so we can detect the end
    piList = []
    for i in range(len(sentence)):
        word = sentence[i]
        if word in modifiedWordDict:
            word = word
        else:
            word = "#UNK#"
            
        piLayer = {}
        if i == 0:
            for tag in all_tags:
                if giantTransitionDict.get("START" + " " + tag) != None and giantEmissionDict.get(word + " " + tag) != None:
                    pi = giantTransitionDict["START" + " " + tag]*giantEmissionDict[word + " " + tag]
                    piLayer[tag] = pi
            piList.append(piLayer)
        
        elif i == 1:
            previousLayer = piList[len(piList)-1]
            for tag in all_tags:
                tempLayer = {}
                for previous_tag in previousLayer:
                    if giantSecondTransitionDict.get("START" + " " + previous_tag + " " + tag) != None and giantEmissionDict.get(word + " " + tag) != None:
                        pi = previousLayer[previous_tag]*giantSecondTransitionDict["START" + " " + previous_tag + " " + tag]*giantEmissionDict[word + " " + tag]
                        tempLayer[previous_tag] = pi
                if len(tempLayer) != 0:
                    maxPiTag = max(tempLayer, key=tempLayer.get) 
                    maxPiScore = max(tempLayer.values())
                    piLayer[tag] = maxPiScore
            piList.append(piLayer)
            
        else:
            print(piList)
            
            previousLayer = piList[len(piList)-1]
            previousPreviousLayer = piList[len(piList)-2]
            for tag in all_tags:
                tempLayer2 = {}
                for previous_previous_tag in previousPreviousLayer:
                    tempLayer = {}
                    for previous_tag in previousLayer:
                        if giantSecondTransitionDict.get(previous_previous_tag + " " + previous_tag + " " + tag) != None and giantEmissionDict.get(word + " " + tag) != None and giantTransitionDict.get(previous_previous_tag + " " + tag):
                            pi = previousLayer[previous_tag]*giantSecondTransitionDict[previous_previous_tag+ " " + previous_tag + " " + tag]*giantEmissionDict[word + " " + tag]
                            tempLayer[previous_tag] = pi
                    

                        

            
            
    
# second_order_viterbi(["The", "tuna", "and", "wasabe", "potatoes", "are", "excellent", "."])
second_order_viterbi(["AVOID", "THAT", "PLACE"])

[{'O': 0.10952242946302841, 'B-negative': 0.0019577571933592874, 'B-positive': 0.010624504726325918, 'B-neutral': 0.0009040105193951349}, {'O': 0.009895079731010215, 'B-negative': 0.0008096240992849406, 'B-positive': 0.004392219908614064, 'B-neutral': 0.00024221306515862052, 'I-negative': 0.0001000958565025801, 'I-positive': 0.0012160526263888527, 'I-neutral': 1.9652402595546413e-05}]
[{'O': 0.10952242946302841, 'B-negative': 0.0019577571933592874, 'B-positive': 0.010624504726325918, 'B-neutral': 0.0009040105193951349}, {'O': 0.009895079731010215, 'B-negative': 0.0008096240992849406, 'B-positive': 0.004392219908614064, 'B-neutral': 0.00024221306515862052, 'I-negative': 0.0001000958565025801, 'I-positive': 0.0012160526263888527, 'I-neutral': 1.9652402595546413e-05}]
